In [ ]:
!unzip /content/drive/MyDrive/archive.zip

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
#Data Folders
train_data_dir = '/content/tomato/train'
test_data_dir = '/content/tomato/val'

In [ ]:
#Set batch size and image size
batch_size = 16
img_height = 150
img_width = 150

In [ ]:
#Data generators for the images
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1.0/255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

In [ ]:
#CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')
])

In [ ]:
#Compile
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
#Train
model.fit(
    train_generator,
    steps_per_epoch=312,
    epochs=10,
    validation_data=test_generator
)

In [ ]:
# Save the model
model.save('plant_leaf_disease_model.h5')

In [ ]:
!pip install gradio

In [ ]:
import gradio as gr
import tensorflow as tf
import numpy as np

# Load the trained model
model = tf.keras.models.load_model('/content/plant_leaf_disease_model.h5')

# Define a function to make predictions
def classify_leaf(image):
    # Preprocess the input image
    img = tf.image.resize(image, (150, 150))
    img = tf.keras.preprocessing.image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = img / 255.0  # Normalize the image

    # Make a prediction
    predictions = model.predict(img)
    class_id = np.argmax(predictions)
    confidence = predictions[0][class_id]

    class_labels = ["Bacterial Spot", "Early Blight", "Late Blight", "Leaf Mold", "Septoria Leaf Spot", "Spider Mites", "Target Spot", "Yellow Leaf Curl Virus", "Mosaic Virus", "Healthy"]

    return f"Predicted Disease:  {class_labels[class_id]}.\nConfidence: {confidence:.2f}"

# Create a Gradio interface
iface = gr.Interface(
    fn=classify_leaf,
    inputs=gr.inputs.Image(type="pil", label="Upload a plant leaf image-"),
    outputs="text"
)

# Launch the interface
iface.launch(share=True)